In [1]:
import pytorch_lightning as L 
import re
import torch
import torch.nn as nn

In [10]:
class reversegrad(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, x):
        return x 
    
    def backward(self, grad_output):
        return -grad_output.neg()



In [13]:
from torch.autograd import Function

class GradientReversal(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.save_for_backward(x, alpha)
        return x
    
    @staticmethod
    def backward(ctx, grad_output):
        grad_input = None
        _, alpha = ctx.saved_tensors
        if ctx.needs_input_grad[0]:
            grad_input = - alpha*grad_output
        return grad_input, None
revgrad = GradientReversal.apply

In [14]:
class GradientReversal(nn.Module):
    def __init__(self, alpha):
        super().__init__()
        self.alpha = torch.tensor(alpha, requires_grad=False)

    def forward(self, x):
        return revgrad(x, self.alpha)

In [2]:
a = torch.randn((5, 3))
a = a.unsqueeze(-1)
a.shape

torch.Size([5, 3, 1])

In [20]:
a = reversegrad() 
b = torch.tensor([1,2,3]).float()
b.requires_grad = True 
c = torch.tensor([4,5,6]).float()
c.requires_grad = True
d = torch.tensor([7,8,9]).float()
d.requires_grad = True
e = b*c 
f = e - d 
g = f.sum() 
g.backward() 
c.grad

tensor([1., 2., 3.])

In [19]:
a = GradientReversal(1) 
b = torch.tensor([1,2,3]).float()
b.requires_grad = True 
c = torch.tensor([4,5,6]).float()
c.requires_grad = True
d = torch.tensor([7,8,9]).float()
d.requires_grad = True
c1 = a(c)
e = b*c1 
f = e - d 
g = f.sum() 
g.backward() 
c.grad

tensor([-1., -2., -3.])

In [20]:
b = a.expand((5, 3, 2))
b.shape, len(b), b.view((30,))

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [21]:
b.reshape((30))

tensor([ 0.6858,  0.6858, -0.9801, -0.9801,  1.2272,  1.2272,  1.2645,  1.2645,
         1.1423,  1.1423, -0.8687, -0.8687,  0.7178,  0.7178,  1.3745,  1.3745,
        -0.9898, -0.9898,  0.1714,  0.1714, -0.0860, -0.0860,  0.1326,  0.1326,
        -1.2204, -1.2204, -0.9274, -0.9274,  2.2042,  2.2042])

In [6]:
s = "abcABCttt%F^S$"
re.sub("[^a-z,A-Z]", "", s)

'abcABCtttFS'

In [10]:
f = open("./utils.py")
for line in f:
    pass
    # print(line.strip())
print(f.readline())

In [21]:
import re
def parseline(line):
    try:
        line = line.decode("utf-8").strip()
    except:
        return None 
    if line.startswith(">"):
        return line 
    # if len(line) == 0:
    #     return "!eof"
    line = re.sub("[^a-z,A-Z]", "", line) 
    return line 

def readFasta(path, to_upper=True, truclength=1500):
    with open(path, "rb") as f:
        line = None
        seq = []  
        for t in f:
            
            # print(t) 
            t = parseline(t)
            # print(t is None)
            if t is None:
                line = None 
                continue
            if line is None:
                if not t.startswith(">"):
                    continue
                else:
                    line = t 
                    continue 
            if t.startswith(">"):
                s = "".join(seq)
                seq = [] 
                # print(s)
                if len(s) > 0:
                    if to_upper:
                        s = s.upper() 
                    if truclength is not None:
                        s = s[:truclength]
                    yield line[1:], s
                line = t
            
            seq.append(t)



In [22]:
# f = open("/home/tyfei/datasets/ion_channel/Interprot/ion_channel/0.99/IPR025692.fasta.0.99.fas", "")

for i in readFasta("/home/tyfei/datasets/ion_channel/Interprot/ion_channel/0.99/IPR025692.fasta.0.99.fas"):
    print(i)

('A0A010SQC8|IPR025692(491...809)|Potassium transporter KefA', 'MPTLRSFFAIALLGLSLSVGSLQAAEPPSAETVQKSLDKIAERKLPEADQKALQLVLQQTLTQLSNRDDYEKRLNDLKQQLSTAPKQNIENQRELERLKASKAVPVEQRYANLSVPQLEQMLTERSTQQSDLQKALADANSLIITAQTRPERAQAEISTSQTRILQINGILKLGRDNGKALTNEQRDTLNAELAALNALIPLRRQELAGNSQLQDLGNSQHDLLLEKTARMEREIQDLQTLINQKRLAQSQQTVTQQSIEAQKAGGSNLLATESAANLKLSDYLLKSTDRLNELTQQNLQTKQQLDSVTQSDAALDEQISVLKGSLLLSKILYKQKQALPRLTLDRNLANDIADIRLYQFEVNQQRELISTPGAYVDNLLASQPPDQVTPQLRKNLLDLAVTRADLLERLSRELSALLNESITLQLNQKQLLTTAQNLRATLDEQMFWIPSNKPLDFEWMRGVPDRLEKQVVTLPWASSLSELADGLTQRPLLFLPLVLLIGFLVWRRKYLYARLNKVHQDIGHFKRDSQWHTPQAILINVLLAMPVSLALALCGFALQIDARGQNANLGAALLQIAQAWLVFYTAYRILAPGGVAQLHFRWERPQVEFLRGWIRRLGLVVLALVAVVAVAELQPAALTDDVIGIAVVLTCYALMAWLLSRLLINSPTHNSASLFRKAVGVLFTVLPIALFIAVCFGYFYTALKLSDRLINTLYLLMFWLIIEATFVRGLAVAARRLAYQRALTKRQAAKEAGDGEAVVEEPTLDIEKVNEQSLRLIRLALLGGFIAALYWVWKDLISVFSYLDNITLYEYTSGVGANMSMVPISIGDMLGALIIIGITVALARNLPGLLEVLVLSKLDLAQGSAYATTTLLTYVIAGVGFVSTLSTLGVSWDKLQWLVAALSVGLGFGMQEIFANFISGIMILFERPVRIGDTIT

In [10]:
!head /home/tyfei/datasets/ion_channel/Interprot/ion_channel/0.99/IPR025692.fasta.0.99.fas

>A0A010SQC8|IPR025692(491...809)|Potassium transporter KefA
MPTLRSFFAIALLGLSLSVGSLQAAEPPSAETVQKSLDKIAERKLPEADQKALQLVLQQTLTQLSNRDDYEKRLNDLKQQ
LSTAPKQNIENQRELERLKASKAVPVEQRYANLSVPQLEQMLTERSTQQSDLQKALADANSLIITAQTRPERAQAEISTS
QTRILQINGILKLGRDNGKALTNEQRDTLNAELAALNALIPLRRQELAGNSQLQDLGNSQHDLLLEKTARMEREIQDLQT
LINQKRLAQSQQTVTQQSIEAQKAGGSNLLATESAANLKLSDYLLKSTDRLNELTQQNLQTKQQLDSVTQSDAALDEQIS
VLKGSLLLSKILYKQKQALPRLTLDRNLANDIADIRLYQFEVNQQRELISTPGAYVDNLLASQPPDQVTPQLRKNLLDLA
VTRADLLERLSRELSALLNESITLQLNQKQLLTTAQNLRATLDEQMFWIPSNKPLDFEWMRGVPDRLEKQVVTLPWASSL
SELADGLTQRPLLFLPLVLLIGFLVWRRKYLYARLNKVHQDIGHFKRDSQWHTPQAILINVLLAMPVSLALALCGFALQI
DARGQNANLGAALLQIAQAWLVFYTAYRILAPGGVAQLHFRWERPQVEFLRGWIRRLGLVVLALVAVVAVAELQPAALTD
DVIGIAVVLTCYALMAWLLSRLLINSPTHNSASLFRKAVGVLFTVLPIALFIAVCFGYFYTALKLSDRLINTLYLLMFWL


In [ ]:
class SeqDataset(Dataset):
    def __init__(self, seq, label):
        self.seq = torch.tensor(seq).long()
        self.label = torch.tensor(label).long()

    def __len__(self):
        return self.seq.shape[0]

    def __getitem__(self, idx):
        
        return self.seq[idx], self.label[idx]
    


class SeqdataModule(L.LightningDataModule):
    def __init__(self, batch_converter, ) -> None:
        super().__init__()